## Project Overview

### Problem Statement
In today's digital world, understanding how Reddit communities function is crucial for moderators, users, and researchers...

### Data Collection Overview
- **Tools**:
  - PRAW (Python Reddit API Wrapper)
  - BeautifulSoup / Scrapy
  
- **Data Points to Collect**:
  - **Posts**: Title, content, upvotes...
  - **Comments**: Content, upvotes...
  
### Solution Approach
1. **Sentiment & Engagement Analysis**
   - Visualizations using Matplotlib and Seaborn...
   
2. **Correlation Analysis**
   - Apply classification algorithms using scikit-learn...

### Expected Deliverables
- Insight Report
- Actionable Recommendations

## Setup Environment

### Purpose
This section prepares our Google Colab environment for the Reddit Communities analysis project as outlined in our team's proposal. We'll install the necessary Python libraries to handle data collection, processing, analysis, and visualization.

### Key Libraries
- PRAW: For accessing the Reddit API
- pandas: For data manipulation and analysis
- numpy: For numerical computing
- matplotlib and seaborn: For data visualization
- nltk: For natural language processing and sentiment analysis
- scikit-learn: For machine learning tasks

### Alignment with Project Goals
These libraries support our objectives of:
1. Analyzing moderation strategies
2. Predicting post impact
3. Visualizing Reddit community interactions

### Installation Code
Run the following cell to install the required libraries:


In [1]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn
import nltk
%pip install praw
import praw
from datetime import datetime


Note: you may need to restart the kernel to use updated packages.


# Praw API Config

This block of code create tje connection with the reddit application and by authenticating.

In [2]:
def setup_reddit_api():
    return praw.Reddit(
        client_id="nrakGjG_wnBE_5UdcHNJoQ",
        client_secret="qmGr1q_4pGIBR0pYJE8cyhUbTbdX2w",
        user_agent="LittleCheesyExplorers/1.0 (Reddit Communities Analysis Project)"
    )

reddit = setup_reddit_api()

print(reddit.user.me())
subreddit = reddit.subreddit("AskReddit")
print(subreddit.display_name)
print(subreddit.title)

None
AskReddit
Ask Reddit...


# Collection Methods

# Testing Collection Methods

# CSV Creation and Storing
In this Code Block we will be creating a csv file and storing all our scraped data.

In [4]:
# List of subreddits to analyze
subreddit_list = ["AskReddit", "news", "funny", "gaming", "todayilearned"]

# Collect data for each subreddit
for subreddit_name in subreddit_list:
    # Collect and save posts
    posts_df = collect_subreddit_posts(subreddit_name, post_limit=50)
    
    # Collect and save comments for the first 5 posts
    for post_id in posts_df['post_id'][:5]:
        collect_comments(post_id, comment_limit=50)
    
    # Collect and save subreddit rules
    collect_subreddit_rules(subreddit_name)

print("Data collection complete. CSV files have been created.")

NameError: name 'collect_subreddit_posts' is not defined